In [1]:
import os
os.chdir("/private/GeoSTHN")
import json
import re

def calculate_average_test_mrr(json_file_path, s, e):
    """
    计算JSON结果文件中所有test mrr的平均值
    
    Args:
        json_file_path: JSON结果文件路径
    
    Returns:
        float: test mrr的平均值
    """
    with open(json_file_path, 'r') as f:
        results = json.load(f)
    
    test_mrr_values = []
    
    for result in results:
        test_mrr_str = result.get("test mrr", "")
        
        # 使用正则表达式提取数值
        # 格式类似 " 0.7952 +-  0.1017"
        match = re.search(r'(\d+\.\d+)', test_mrr_str.strip())
        if match:
            test_mrr_value = float(match.group(1))
            test_mrr_values.append(test_mrr_value)
            # print(f"Run {len(test_mrr_values)-1}: {test_mrr_value:.4f}")
    
    if test_mrr_values:
        average_mrr = sum(test_mrr_values) / len(test_mrr_values)
        if s is not None :
            if s < 0:
                s = max(0, len(test_mrr_values) + s)
        
            if e is None:
                e = len(test_mrr_values)
            elif e < 0:
                e = max(0, len(test_mrr_values) + e)
            test_mrr_values = test_mrr_values[s:e]
        else:
            s = 0
            e = len(test_mrr_values)
        average_mrr = sum(test_mrr_values) / len(test_mrr_values)
        std_dev = (sum((x - average_mrr) ** 2 for x in test_mrr_values) / len(test_mrr_values)) ** 0.5
        
        print(f"\n📊 统计结果:")
        print(f"总运行次数: {len(test_mrr_values)}")
        print(f"考虑范围: [{s}, {e})")
        print(f"Test MRR 平均值: {average_mrr:.4f}")
        print(f"标准差: {std_dev:.4f}")
        print(f"最高值: {max(test_mrr_values):.4f}")
        print(f"最低值: {min(test_mrr_values):.4f}")
        
        return average_mrr
    else:
        print("❌ 未找到有效的test mrr数据")
        return None

def calculate_stats_by_seed(json_file_path):
    """
    按照seed分组计算统计数据（如果有多个seed的话）
    """
    with open(json_file_path, 'r') as f:
        results = json.load(f)
    
    seed_results = {}
    
    for result in results:
        seed = result.get("seed", 1)
        test_mrr_str = result.get("test mrr", "")
        
        match = re.search(r'(\d+\.\d+)', test_mrr_str.strip())
        if match:
            test_mrr_value = float(match.group(1))
            
            if seed not in seed_results:
                seed_results[seed] = []
            seed_results[seed].append(test_mrr_value)
    
    print(f"\n📈 按Seed分组统计:")
    overall_values = []
    
    for seed, values in seed_results.items():
        avg = sum(values) / len(values)
        std = (sum((x - avg) ** 2 for x in values) / len(values)) ** 0.5
        print(f"Seed {seed}: {avg:.4f} ± {std:.4f} (n={len(values)})")
        overall_values.extend(values)
    
    if overall_values:
        overall_avg = sum(overall_values) / len(overall_values)
        overall_std = (sum((x - overall_avg) ** 2 for x in overall_values) / len(overall_values)) ** 0.5
        print(f"\n🎯 总体平均: {overall_avg:.4f} ± {overall_std:.4f} (n={len(overall_values)})")
        
        return overall_avg, overall_std
    
    return None, None

In [3]:
# exper_name = "hetero_sthn_rgfm"
# exper_name = "rgfm-htero"
exper_name = "rgfm-htero-newtrain"
# exper_name = "abl_no_rie"

datasets = ["thgl-forum-subset", "thgl-github-subset","thgl-myket-subset","thgl-software-subset"]
# datasets = ["thgl-myket-subset"]

for dataset in datasets:
    json_file_path = f"exper/{exper_name}/{dataset}/result/{dataset}_results.json"
    
    print(f"\n🔍 处理数据集: {dataset}")
    
    # 计算平均test mrr
    calculate_average_test_mrr(json_file_path,s=0,e=None)
    # calculate_average_test_mrr(json_file_path,s=None,e=None)


🔍 处理数据集: thgl-forum-subset

📊 统计结果:
总运行次数: 10
考虑范围: [0, 10)
Test MRR 平均值: 0.8389
标准差: 0.0253
最高值: 0.8765
最低值: 0.8022

🔍 处理数据集: thgl-github-subset

📊 统计结果:
总运行次数: 2
考虑范围: [0, 2)
Test MRR 平均值: 0.8056
标准差: 0.0118
最高值: 0.8174
最低值: 0.7938

🔍 处理数据集: thgl-myket-subset

📊 统计结果:
总运行次数: 2
考虑范围: [0, 2)
Test MRR 平均值: 0.9295
标准差: 0.0181
最高值: 0.9476
最低值: 0.9115

🔍 处理数据集: thgl-software-subset


AttributeError: 'str' object has no attribute 'get'